# iris CART using DecisionTree.jl
Loading packages that we will use

In [1]:
using DecisionTree
using Random, Statistics
using DataFrames

Loading Data (Iris dataset)

In [2]:
X, y = load_data("iris")
DF1 = DataFrame(X, :auto)
DF2 = DataFrame("Type" => y )
DF = DataFrame(x1 = float.(DF1[!,1]),
x2 = float.(DF1[!,2]),
x3 = float.(DF1[!,3]),
x4 = float.(DF1[!,4]),
y = string.(DF2[!,1]));
Matrix(DF[!,1:4]);


Now using pycall to split data into training and testing sets

In [72]:
using PyCall
ModelSelection = pyimport("sklearn.model_selection")
X_train, X_test, y_train, y_test =
 ModelSelection.train_test_split(Matrix(DF[!,1:4]),DF[!,5]);

Decision Tree

In [4]:
model = DecisionTreeClassifier(max_depth = 2);
fit!(model, X_train, y_train);
#Printting the tree
print_tree(model)

Feature 3 < 2.45 ?
├─ Iris-setosa : 34/34
└─ Feature 3 < 4.85 ?
    ├─ Iris-versicolor : 36/37
    └─ Iris-virginica : 39/41


In [5]:
train = [X_train y_train]
# view decision node data subset

train_R = train[train[:, 4] .> 0.8, :];


Ready to make some predictions

In [6]:
y_hat = predict(model, X_test)
#and checking the accuracy
accuracy = mean(y_hat .== y_test)

0.8947368421052632

Let's see where the model get confussed

In [8]:
DecisionTree.confusion_matrix(y_test,y_hat)

3×3 Matrix{Int64}:
 16   0  0
  0  10  2
  0   2  8

Classes:  ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.8947368421052632
Kappa:    0.8389830508474576

Display results

In [13]:
check = [y_hat[i] == y_test[i] for i in 1:length(y_hat)]
check_display = [y_hat y_test check]

38×3 Matrix{Any}:
 "Iris-virginica"   "Iris-virginica"    true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-virginica"   "Iris-versicolor"  false
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-setosa"      "Iris-setosa"       true
 ⋮                                     
 "Iris-virginica"   "Iris-virginica"    true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-virginica"   "Iris-virginica"    true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-setosa"      "Iris-setosa"       true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-versicolor"  "Iris-versicolor"   true
 "Iris-setosa"      "Iris-setosa"       tr

Display probability of each prediction

In [20]:
prob = predict_proba(model, sort(X_test,dims = 1))
display(prob)

38×3 Matrix{Float64}:
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 1.0  0.0        0.0
 ⋮               
 0.0  0.972973   0.027027
 0.0  0.972973   0.027027
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122
 0.0  0.0487805  0.95122

We can improve our predicition but it will come with the Bias-variance tradeoff.\
Ensemble learning for improve decision tree,they are two different techniques, one is Bagging, the other is Boosting.  

## Random Forest
Example of the Bagging technique, it reduces the variance of the model (Increase independecy of the features or Increase number of models) but you get slightly higher bias.

In [57]:
#Creating a random forest with 20 trees
model1 = RandomForestClassifier(n_trees = 20);
fit!(model1, X_train, y_train)
#making predicitions
ŷ = predict(model1, X_test)
#checking accuracy 
accuracy1 = mean(y_test .== ŷ)
#confusion matrix
@show DecisionTree.confusion_matrix(y_test, ŷ)
#checking the probability
prob1 = predict_proba(model1, sort(X_test, dims = 1))

3×3 Matrix{Int64}:
 16   0  0
  0  10  2
  0   1  9

DecisionTree.confusion_matrix(y_test, ŷ) = Classes:  ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9210526315789473
Kappa:    0.879492600422833


38×3 Matrix{Float64}:
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 1.0  0.0   0.0
 ⋮          
 0.0  1.0   0.0
 0.0  0.55  0.45
 0.0  0.1   0.9
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0
 0.0  0.0   1.0

## AdaBoost
Technique of Boosting in ensemble learning where usually lower the bias

In [58]:
model2 = AdaBoostStumpClassifier(n_iterations = 20);
fit!(model2, X_train, y_train)
#making predicitions
ŷ = predict(model2, X_test)
#checking accuracy 
accuracy1 = mean(y_test .== ŷ)
#confusion matrix
@show DecisionTree.confusion_matrix(y_test, ŷ)
#checking the probability
prob1 = predict_proba(model2, sort(X_test, dims = 1))


3×3 Matrix{Int64}:
 16   0   0
  0  11   1
  0   0  10

DecisionTree.confusion_matrix(y_test, ŷ) = Classes:  ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9736842105263158
Kappa:    0.9598308668076109


38×3 Matrix{Float64}:
 0.611055  0.29135   0.0975955
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 0.611055  0.338964  0.0499806
 ⋮                   
 0.0       0.61114   0.38886
 0.0       0.49819   0.50181
 0.0       0.462856  0.537144
 0.0       0.400216  0.599784
 0.0       0.400216  0.599784
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144
 0.0       0.355856  0.644144

## Using SKlearn with PyCall

In [59]:
using MLBase

In [75]:
LabelMap = labelmap(DF[:,5])
y_train1 = labelencode(LabelMap, y_train)
y_test1 = labelencode(LabelMap, y_test);


In [93]:
np = pyimport("numpy")
skl = pyimport("sklearn")
ModelEnsemble = pyimport("sklearn.ensemble")
SklMtr = pyimport("sklearn.metrics")
jol = pyimport("joblib")

PyObject <module 'joblib' from 'C:\\Users\\PC\\.julia\\conda\\3\\lib\\site-packages\\joblib\\__init__.py'>

In [92]:
model3 = ModelEnsemble.RandomForestClassifier()
model3.fit(X_train,y_train1)
y_pred = model3.predict(X_test)
model3.score(X_test,y_test1)
SklMtr.confusion_matrix(y_test1, y_pred)

3×3 Matrix{Int64}:
 16   0  0
  0  11  1
  0   1  9

Saving model3

In [96]:
joblib_file = "IrisDetection.joblib"
jol.dump(model3, joblib_file)

1-element Vector{String}:
 "IrisDetection.joblib"